# Proyecto 1: Interpolación, Integracion y raíces de Ecuaciones


Función asignada: *Polinomios de Legendre asociados* 

https://es.wikipedia.org/wiki/Polinomios_asociados_de_Legendre

Los polinomios asociados de Legendre son las soluciones canónicasde la **ecuación de Legendre**.

<center>$(1-x^{2})y'' - 2xy' + (l[l+1]-\frac{m^{2}}{1-x^{2}})y = 0$</center>

O de forma equivalente:

<center>$([1-x^{2}]y')' + (l[l+1]-\frac{m^{2}}{1-x^{2}})y = 0$</center>

**$l: $Grado del polinomio, $m: $ Orden del polinomio**

Esta ecuación tiene soluciones distintas de 0 que son no singulares en **[-1,1]** sólo si ℓ y m son enteros con 0 ≤ m ≤ ℓ. 

> Cuando m es cero y ℓ entero, estas funciones son idénticas a los **polinomios de Legendre. **

En general, cuando ℓ y m son enteros, las soluciones regulares a veces son llamadas "polinomios asociados de Legendre", incluso cuando estas no son polinomios en el caso de que m sea impar. 


## Polinomios ortogonales


Los polinomios ortogonales son conjuntos de polinomios que forman una base ortogonal de cierto espacio de Hilbert. Los polinomios ortogonales son importantes porque aparecen en la teoría de ecuaciones diferenciales, muy especialmente en la teoría de Sturm-Liouville, la teoría de espacios de Hilbert, la teoría de la aproximación de funciones y la mecánica cuántica.

Asumiendo que $0\leq m \leq l$ se satisface  la condición de ortogonalidad para un m fijo:


<center>$\int _{-1}^{1} P_{k}^{m}P_{l}^{m}dx = \frac{2(l+m)!}{(2l+1)(l-m)!}\delta_{k,l}$</center>

Donde $\delta_{k,l}$ es el delta de kronecher.

## Relaciones de recurrencia 

En matemática, una relación de recurrencia es una ecuación que define una secuencia recursiva; cada término de la secuencia es definido como una función de términos anteriores.

Los polinomios de Legendre pueden construirse usando las tres relaciones de recurrencia

$(n+1)P_{n+1} = (2n+1)xP_{n}- nP_{n-1}$

y

$\frac{x^{2}-1}{n} \frac{d}{dx}P_n = xP_n - P_{n-1} $



## Aplicaciones de polinomios de Legendre en Física

Los polinomios de Legendre, igual que los de Hermite y Laguerre, son útiles en ramas de la Física y en el cálculo numérico ya que permiten el cómputo de integrales definidas sin necesidad de usar fórmulas analíticas, tan sólo fijando como intervalo de integración [ -1 ; +1] (con el correspondiente cambio de variable). Esto es especialmente interesante en programas de cómputo que tratan de resolver una integral definida.


$ \frac{\partial u}{\partial t}
   = h^2 \left( \frac{\partial^2 u}{\partial x^2}
      + \frac{\partial^2 u}{\partial y^2}
      + \frac{\partial^2 u}{\partial z^2} \right)$
      
      
$\frac{du}{dt} and \frac{d^2 u}{dx^2} $

In [36]:
import numpy as np
from sympy import *
import matplotlib.pyplot as plt
from scipy.misc import comb
%matplotlib inline

## 1) Interpolación
### 1.1 ) Elabórese un programa para calcular su polinomio ortogonal $P_n(x)$ usando las relaciones de recurrencia

In [59]:
def Pn(n):
    x = symbols('x')
    Ps = [0, x]
    if n == 0:
        return 1
    elif n == 1:
        return x
    else:
        h = n-1    
        p = ( (2.*h +1)*x*Pn(h)- h*Pn(h-1))/(h+1.) 
        return p

In [60]:
grado = 3
P = simplify(Pn(grado))
print "Pn de grado", grado, ": ",P
#P.subs(x,3)

Pn de grado 3 :  x*(2.5*x**2 - 1.5)


### 1.2) Dentro del intervalo de definición escójase ne = 11 nodos equidistantes xe(i), i:1,..,11 y encuéntrese los valores

In [62]:
f = lambdify(x, P)
xl = np.linspace(-1,1,11)
yl = f(xl)

print "xe: ",xl
print "\nye: ",yl

xe:  [-1.  -0.8 -0.6 -0.4 -0.2  0.   0.2  0.4  0.6  0.8  1. ]

ye:  [-1.   -0.08  0.36  0.44  0.28 -0.   -0.28 -0.44 -0.36  0.08  1.  ]


###  1.3) Elabore 3 programas para calcular los polinomios interpolantes en la base de algoritmos: Lagrange global, segmentaria lineal y parabólica

In [11]:
#Stuff
def legendre_rodrigues(n):
    "Retorna polinomio de legendre grado n"
    suma = 0
    x = symbols('x')
    polinomio = 0 #POLINOMIO SOLUCION
    for k in range(n+1):
        tmp = (comb(n,k)**2)*((x+1.)**(n-k))*(x-1.)**k
        suma = suma+tmp
    p = (1.0/(2.**n))*suma
    return p

def funcion_analitica(xi,n):
    """Retorna polinomio de legendre de grado n evaluado en x_i."""
    x = symbols('x')
    return  legendre_rodrigues(n).evalf(subs={x:xi})